In [23]:
# Install some packages
!pip install rasterio
!pip install earthpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.8 MB/s eta 0:00:00


In [5]:
# Import lib packages
import pandas as pd
import numpy as np
import keras
from keras import Sequential
from keras.Layers import Conv1D,MaxPooling1D,Dense,Dropout,Flatten,Input,GlobalMaxPooling1D
from keras.callbacks import EarlyStopping
from keras import Model
import rasterio
import earthpy.plot as ep
from keras.utils import to_categorial,plot_model,model_to_dot
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import ee
import geemap
from samgeo import SamGeo

In [21]:
Map = geemap.Map()

# Create point of interest (poi)
poi = ee.Geometry.Point (-74.62758, 40.67981)

# Create image collection
img_col = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterBounds(poi)
    .filterDate('2008-01-01','2018-01-01')
    .filter(ee.Filter.listContains("system:band_names","N"))
    )

# Select the first image
image = img_col.first()

# Add image to layer
Map.addLayer(image,{},'Naip')
Map.centerObject(poi  ,16)
Map

Map(center=[40.67981000000001, -74.62758], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…

In [14]:
bbox = Map.user_roi_coords ()
if bbox is None:
  bbox = [-74.62758, 40.67981,-74.64813896488816,40.672844244342286]

In [22]:
# Export to cloud storage or an earth engine assest
image_task = ee.batch.Export.image.toDrive(**{
    'image': image,
    'description': 'naip',
    'folder': 'sam-example',
    'fileNamePrefix': 'naip',
    'scale':1,
    'region':bbox
})

image_task.start()

EEException: ignored